In [83]:
import seaborn as sns
import matplotlib.pyplot as plt

import findspark
findspark.init()
from pyspark.sql.functions import isnull, col
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer, StandardScaler, Imputer, YeoJohnsonTransformer
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.types import *
from pyspark.sql.functions import udf

ImportError: cannot import name 'YeoJohnsonTransformer' from 'pyspark.ml.feature' (C:\Spark3\python\pyspark\ml\feature.py)

In [3]:
spark = SparkSession.builder.master('local') \
    .appName("ZMClustering") \
    .config("spark.driver.cores", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()
sc = spark.sparkContext

In [6]:
data = (spark.read
        .csv("C:\Repositories\MineralogicData_Clustering\data\centroid_PINiCo2.csv", header=True, inferSchema=True)
        .repartition('sample')
)

data = (
    data
 .withColumnRenamed('Particle ID', 'particleID')
 .withColumnRenamed('Grain ID', 'grainID')
 .withColumnRenamed('Mineral Classification', 'mineral')
 .withColumnRenamed('Spectrum Counts', 'spectrum_cts')
 .withColumnRenamed('Area µm²', 'area_um2')
 .drop('_c0')
)

In [ ]:
# data.write.mode('overwrite').format("csv").save("C:\Repositories\MineralogicData_Clustering\data\parquet") #not working


In [28]:
data.show(5)

+-----+----------+------+-------+-------+------------+--------+---------+--------+-----+-----+---+---+---+-----+---+---+---+---+---+----+---+---+---+---+---+---+---+---+
|index|particleID|sample|grainID|mineral|spectrum_cts|Porosity|Roughness|area_um2|   Ti|    O|  C| Fe| Al|   Si| Ni| Mn| Co| Mg|  S|   V| Sc| Cu| Cr| Ca| Ba| Na|  K| Zn|
+-----+----------+------+-------+-------+------------+--------+---------+--------+-----+-----+---+---+---+-----+---+---+---+---+---+----+---+---+---+---+---+---+---+---+
|    0|   1594685| 53x45|1594685|Anatase|        3277|    5.08|     1.79|  3310.1|62.01|37.24|0.0|0.0|0.0|  0.0|0.0|0.0|0.0|0.0|0.0|0.74|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|
|    1|   1594697| 53x45|1594697|Anatase|        3297|     0.0|     1.04|     0.8|42.08| 45.7|0.0|0.0|0.0|11.65|0.0|0.0|0.0|0.0|0.0|0.57|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|
|    2|   1594698| 53x45|1594698|Anatase|        3491|     0.0|     1.15|   10.37|42.68|50.13|0.0|0.0|0.0| 7.19|0.0|0.0|0.0|0.0|0.0| 0.0|0.0|0.0|0.0|0

In [8]:
features = ['Ti', 'O', 'C', 'Fe', 'Al', 'Si', 'Ni', 'Mn', 'Co', 'Mg', 
            'S', 'V', 'Sc', 'Cu', 'Cr', 'Ca', 'Ba', 'Na', 'K', 'Zn']

In [82]:
print(data_scaled.select('features_unscaled').take(1)[0].features_unscaled.toArray())

[62.01 37.24  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.74
  0.    0.    0.    0.    0.    0.    0.    0.  ]


In [ ]:
# def centred_log_ratio(data, inputCol, outputCol):
#     for i in [0,1,2,3,4]:
#         data.select(inputCol)[i]

#     return data

# centred_log_ratio(data_scaled, inputCol='features_unscaled')

In [23]:
vector = VectorAssembler(inputCols=features, outputCol='features_unscaled')
CLR = udf(lambda data: centred_log_ratio(data, inputCols='features_unscaled', outputCol='features_unscaled'), returnType=DoubleType())
# imputer = Imputer(inputCols = ['features_unscaled'], outputCols=['features_unscaled'], strategy=0)
yeo = YeoJohnsonTransformer(inputCol="features", outputCol="transformed_features")
scaler = StandardScaler(inputCol = "features_unscaled", outputCol = "features_scaled")

pipeline = Pipeline(stages = [
    vector, scaler
])

data_scaled = pipeline.fit(data).transform(data)

In [29]:
data_scaled.show(5, truncate = False)

+-----+----------+------+-------+-------+------------+--------+---------+--------+-----+-----+---+---+---+-----+---+---+---+---+---+----+---+---+---+---+---+---+---+---+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|index|particleID|sample|grainID|mineral|spectrum_cts|Porosity|Roughness|area_um2|Ti   |O    |C  |Fe |Al |Si   |Ni |Mn |Co |Mg |S  |V   |Sc |Cu |Cr |Ca |Ba |Na |K  |Zn |features_unscaled                           |features_scaled                                                                                                  |
+-----+----------+------+-------+-------+------------+--------+---------+--------+-----+-----+---+---+---+-----+---+---+---+---+---+----+---+---+---+---+---+---+---+---+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|0    |159468